In [1]:
from mds.database.mongo import GetConfig
from mds.database.minio import GetMinioConfig
from mds.database.config import MONGO_DATABASE, MONGO_COLLECTION
from mds.models.download import Download
from mds.models.dataset import Dataset
from mds.models.project import Project
from mds.models.organization import Organization
from mds.models.user import User

In [2]:
mongo_client = GetConfig()
mongo_db = mongo_client[MONGO_DATABASE]
mongo_collection = mongo_db[MONGO_COLLECTION]

In [3]:
minio_client = GetMinioConfig()

In [17]:
# create a user
user1_data = {
    "@id": "ark:99999/test-user1",
    "name": "Test User1",
    "type": "Person",
    "email": "testuser1@example.org",
    "password": "test1",
    "organizations": [],
    "projects": [],
    "datasets": [],
    "software": [],
    "computations": [],
    "evidencegraphs": []
}

user1 = User(**user1_data)

In [9]:
print(user1.create(mongo_collection))

Success: True	Message: 	StatusCode: 200


In [18]:
# create an org
org_data = {
    "@id": "ark:99999/test-org",
    "name": "test organization",
    "@type": "Organization",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

org = Organization(**org_data)

In [ ]:
print(org.create(mongo_collection))

In [19]:
# create a project
project_data = {
    "@id": "ark:99999/test-org/test-proj",
    "name": "Test Project",
    "@type": "Project",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    },
    "memberOf": {
        "@id": "ark:99999/test-org",
        "name": "test organization",
        "@type": "Organization",
    }
}

proj = Project(**project_data)


In [ ]:
print(proj.create(mongo_collection))

In [20]:
# create a dataset
dataset_data = {
    "@id": "ark:99999/test-org/test-proj/test-data",
    "@type": "Dataset",
    "name": "Test Dataset",
    "owner": {
        "@id": user1_data['@id'],
        "name": user1_data['name'],
        "email": user1_data['email'],
        "@type": "Person"
    }
}

ds = Dataset(**dataset_data)

In [ ]:
print(ds.create(mongo_client))

In [21]:
# create a download
download_data = {
    "@id": "ark:99999/test-org/test-proj/test-data/file1.txt",
    "@type": "DataDownload",
    "name": "Test CSV",
    "encodingFormat": ".csv",
    "encodesCreativeWork": {
        "@id": "ark:99999/test-org/test-proj/test-data",
        "@type": "Dataset",
        "name": "Test Dataset",
    }
}

dload = Download(**download_data)

In [14]:
data_file = open("tests/test-data.csv", "rb")
print(dload.register(mongo_collection, minio_client, data_file))

Success: True	Message: 	StatusCode: 201


## Testing with API

In [15]:
import requests
import json

root_url = "http://localhost:8000/"

In [23]:
# create user
user_create = requests.post(root_url + "user", json=user1_data)

In [ ]:
print(user_create.status)

In [25]:
# create organization
organization_create = requests.post(root_url + "organization", json=org_data)

In [ ]:


dataset_create = requests.post(root_url + f"dataset/", data=json.dumps(dataset_data))
dataset_create.json()